In [3]:
from activation_additions import consts, prompt_utils
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch.nn as nn
import importlib
#import activation_additions.adhoc_actadds

import torch

model_name = "llama-7b"
MODEL_PATH = consts.LLAMA_PATH + consts.llama_relpaths[model_name]
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)
model = LlamaForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map="auto",
    low_cpu_mem_usage=True,
)
model.eval()
gen = model.generate(
    tokenizer("Do you know how the world ends? ", return_tensors="pt").input_ids,
    temperature=0,
    max_new_tokens=20,
)
print(tokenizer.decode(gen[0]))

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/ssd-2/mesaoptimizer/miniconda3/envs/wuschel_env/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s>Do you know how the world ends? 100 years from now, the world will end. The world will end because of the people


In [71]:
import importlib
import activation_additions.adhoc_actadds

# ... make some changes to 'some_library' ...

# Now reload it to incorporate the changes
importlib.reload(activation_additions.adhoc_actadds)

<module 'activation_additions.adhoc_actadds' from '/mnt/ssd-2/mesaoptimizer/wuschel/new_download/activation_additions/activation_additions/adhoc_actadds.py'>

In [5]:
prompt_utils.get_x_vector("hallo","bys",2,2,
    model)

(ActivationAddition(hallo, 2, blocks.2.hook_resid_pre),
 ActivationAddition(bys, -2, blocks.2.hook_resid_pre))

In [63]:
model.model.layers[0].self_attn


LlamaAttention(
  (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
  (rotary_emb): LlamaRotaryEmbedding()
)

In [64]:
activations

tensor([[[-0.0244,  0.0359,  0.0344,  ...,  0.0345,  0.0770,  0.0308],
         [-0.0189,  0.0014,  0.0172,  ...,  0.0203,  0.0617,  0.0399],
         [ 0.0043, -0.0016,  0.0332,  ...,  0.0077,  0.0575,  0.1177]]],
       device='cuda:2')

In [65]:
input_text = "I hate you because"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
remove_all_hooks(model)
forward_vanilla=model.forward(input_ids)["logits"]
layer=10
coeff=1
activations=get_residual_activation(model, tokenizer(" love", return_tensors="pt").input_ids,layer)
register_residual_hook(model, layer, -coeff*activations)
activations=get_residual_activation(model, tokenizer(" hate", return_tensors="pt").input_ids,layer)
#register_residual_hook(model, layer, coeff*activations)
#forward__hooked=model.forward(input_ids)["logits"]


tensor([[    1, 29871,  5360]], device='cuda:0')
tensor([[    1, 29871, 26277]], device='cuda:0')


UnboundLocalError: local variable 'new_output' referenced before assignment

In [12]:
layer=5
coeff=5
remove_all_hooks(model)
activations=get_residual_activation(model, tokenizer(" love", return_tensors="pt").input_ids,layer)
register_residual_hook(model, layer, coeff*activations)
activations=get_residual_activation(model, tokenizer(" hate", return_tensors="pt").input_ids,layer)
register_residual_hook(model, layer, -coeff*activations)


AttributeError: 'LlamaDecoderLayer' object has no attribute 'self_attention'

In [10]:

# Using the generate method
input_text = "I hate you because"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
gen_tokens = model.generate(input_ids, temperature=1, max_new_tokens=20)
generated_text = tokenizer.decode(gen_tokens[0])
print(generated_text)

/mnt/ssd-2/mesaoptimizer/miniconda3/envs/wuschel_env/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s>I hate you because you....---------------


In [64]:
remove_all_hooks(model)
en_tokens = model.generate(input_ids, temperature=1, max_new_tokens=20)
generated_text = tokenizer.decode(gen_tokens[0])
print(generated_text)

gen_tokens = model.generate(input_ids, temperature=1, max_new_tokens=20)
generated_text = tokenizer.decode(gen_tokens[0])
print(generated_text)

gen_tokens = model.generate(input_ids, temperature=1, max_new_tokens=20)
generated_text = tokenizer.decode(gen_tokens[0])
print(generated_text)

<s>I hate you because you are a woman.
I hate you because you are a woman. I hate you because you
<s>I hate you because you are so beautiful.
I hate you because you are so beautiful. I hate you because you
<s>I hate you because you are so beautiful.
I hate you because you are so beautiful. I hate you because you


In [87]:

# Reload the module to get the most recent version of the HookedModel class
importlib.reload(activation_additions.adhoc_actadds)

from activation_additions.adhoc_actadds import HookedModel
hooked_model = HookedModel(model, tokenizer)

In [88]:
ActAds =[prompt_utils.ActivationAddition(coeff=1, act_name="layers.6",prompt=" hate"),
         prompt_utils.ActivationAddition(coeff=-1, act_name="layers.6",prompt=" love")]

In [89]:
#hooked_model.remove_all_hooks()
hooked_model.add_activation_hooks(ActAds)

In [90]:
# Using the generate method
input_text = "Do you know how I feel? I feel"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
gen_tokens = hooked_model.generate(input_ids, temperature=0, max_new_tokens=20)
generated_text = tokenizer.decode(gen_tokens[0])
print(generated_text)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/mnt/ssd-2/mesaoptimizer/miniconda3/envs/wuschel_env/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3067524/1375328358.py", line 4, in <module>
    gen_tokens = hooked_model.generate(input_ids, temperature=0, max_new_tokens=20)
  File "/mnt/ssd-2/mesaoptimizer/wuschel/new_download/activation_additions/activation_additions/adhoc_actadds.py", line 75, in generate
    return self.model.generate(input_ids, **kwargs)
  File "/mnt/ssd-2/mesaoptimizer/miniconda3/envs/wuschel_env/lib/python3.10/site-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/mnt/ssd-2/mesaoptimizer/miniconda3/envs/wuschel_env/lib/python3.10/site-packages/transformers/generation/utils.py", line 1538, in generate
    return self.greedy_search(
  File "/mnt/ssd-2/mesaoptimizer/miniconda3/envs/wuschel_env/li

In [16]:
# Using the generate method
input_text = "Do you know how the world ends? "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
gen_tokens = hooked_model.generate(input_ids, temperature=0, max_new_tokens=20)
generated_text = tokenizer.decode(gen_tokens[0])
print(generated_text)

# Using the forward method (for getting logits or other outputs)
inputs = tokenizer("Hello World", return_tensors='pt')
outputs = hooked_model.forward(**inputs)
logits = outputs.logits


/mnt/ssd-2/mesaoptimizer/miniconda3/envs/wuschel_env/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


<s>Do you know how the world ends? 100 years from now, the world will end. The world will end because of the people


In [4]:
import torch.nn as nn
import torch
def print_layers(model: nn.Module, prefix=""):
    for name, child in model.named_children():
        current_name = f"{prefix}.{name}" if prefix else name
        print(current_name)
        print_layers(child, current_name)

# Example usage
# model = LlamaForCausalLM.from_pretrained(...)
# print_layers(model)


In [7]:
model

NameError: name 'model' is not defined

In [34]:
model{"model.layers.0"}

SyntaxError: invalid syntax (376426420.py, line 1)

In [30]:
print_layers(model)

model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.s

In [18]:
logits.shape

torch.Size([1, 3, 32000])

In [11]:
gen = hooked_model.generate(
    tokenizer("Do you know how the world ends? ", return_tensors="pt").input_ids,
    temperature=0,
    max_new_tokens=20,
)
print(tokenizer.decode(gen[0]))

AttributeError: 'HookedModel' object has no attribute 'generate'

In [5]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [3]:
gen = model.generate(
    tokenizer("def fibonacci(n):", return_tensors="pt").input_ids,
    temperature=0,
    max_new_tokens=80,
)
print(tokenizer.decode(gen[0]))

<s>def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

def fibonacci_iterative(n):
    if n == 0:
        return 0
    elif n ==
